# The Traveling Salesman Problem (TSP)
#### The TSP depicts a salesman who has to visit $n$ cities, returning to it's start  (home)  city, whilst not visiting any of the nodes more than once. 
Store data in an adjacency matrix.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math 
from math import floor
from random import randint
import csv as csv
#to shuffle dataframe
from sklearn.utils import shuffle 
from IPython.display import display, HTML
import scipy 
from scipy.misc import comb # comb(n,k, exact=True)
from math import inf
# from scipy import special
# from scipy.special import comb
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [2]:
city_dist = 'city'
def read_data(fileName):
    df = pd.read_csv(fileName)
    return df
    
def check_packaging(df):
    rows, cols = df.shape #size of the data set
    return (rows, cols)

def data_check(df, n=3):
    df_top_n = df.head(n)
    return (df_top_n)

def check_ns(df):
    ns = df.describe()
    return ns


In [3]:
df_cityDist = read_data('%s.csv'%city_dist)
print("rows(%s) x cols(%s) "%check_packaging(df_cityDist))
print()
print("%s"%data_check(df_cityDist))
print()
print(check_ns(df_cityDist))
print()
num_cities = df_cityDist.shape[0]
# df_cityDist.set_index('city')
print('Number of cities including depot: ', num_cities)

rows(5) x cols(5) 

     0   1    2    3   4
0    0  16  100   14  22
1   16   0   18  100   4
2  100  18    0   18  23

                0           1           2           3          4
count    5.000000    5.000000    5.000000    5.000000   5.000000
mean    30.400000   27.600000   31.800000   27.800000  11.200000
std     39.734116   41.192232   39.117771   40.941422  10.616026
min      0.000000    0.000000    0.000000    0.000000   0.000000
25%     14.000000    4.000000   18.000000    7.000000   4.000000
50%     16.000000   16.000000   18.000000   14.000000   7.000000
75%     22.000000   18.000000   23.000000   18.000000  22.000000
max    100.000000  100.000000  100.000000  100.000000  23.000000

Number of cities including depot:  5



#### Initilaiization of dataframes which are used to account for verticies visited and calculate distance travelled
NOTE: the $0^{th}$ vertex is the start/depot vertex

In [4]:
distances = pd.DataFrame(np.zeros((num_cities,1)))# The distance from Vertex i to the next vertex
possible = pd.DataFrame(np.zeros((num_cities,1)))# Accounts for the vertices that have been visited
possible.loc[:,:] = True
terminal_set = pd.DataFrame(np.zeros((num_cities,1)))# The set/ordering of vertices that are encountered on the route

distances.rename(columns={0: 'distances'}, inplace=True)
possible.rename(columns={0: 'possible'}, inplace=True)
terminal_set.rename(columns={0: 'terminal_set'}, inplace=True)

display(distances)
display(possible)
display(terminal_set)
display(df_cityDist)


,distances
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


,possible
0,True
1,True
2,True
3,True
4,True


,terminal_set
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


,0,1,2,3,4
0,0,16,100,14,22
1,16,0,18,100,4
2,100,18,0,18,23
3,14,100,18,0,7
4,22,4,23,7,0


In [5]:
for i in range(num_cities):
    df_cityDist.iloc[i,i] = inf
df_cityDist

,0,1,2,3,4
0,inf,16.000000,100.000000,14.000000,22.000000
1,16.000000,inf,18.000000,100.000000,4.000000
2,100.000000,18.000000,inf,18.000000,23.000000
3,14.000000,100.000000,18.000000,inf,7.000000
4,22.000000,4.000000,23.000000,7.000000,inf


Psuedo-code for nearest neighbour algorithm:
1. From start node (V0), find the nearest neighbour (Vn). In the zeroth row from col 1:end find the smallest distance.
2. Put the next visited neighbour in the terminal set -> T. Add the new vertes  to the the possible set -> P. (Note: this must be assigned to Vn)
3. Do this while D(V0) = 0

In [6]:
distances = pd.DataFrame(np.zeros((num_cities,1)))# The distance from Vertex i to the next vertex
possible = pd.DataFrame(np.zeros((num_cities,1)))# Accounts for the vertices that have been visited
possible.iloc[:,:] = True
terminal_set = pd.DataFrame(np.zeros((num_cities,1)))# The set/ordering of vertices that are encountered on the route

distances.rename(columns={0: 'distances'}, inplace=True)
possible.rename(columns={0: 'possible'}, inplace=True)
terminal_set.rename(columns={0: 'terminal_set'}, inplace=True)
    
def nearest_neighbour(adj_mat, num_cities, distances, possible,terminal_set):#take in the adjacency matrix
    #define possible marix, distance matric and terminal set 
    count = 0
    new_curr = 0 #depot is the first vertex
    while possible.iloc[0,0]== True: 
        if count < num_cities-1:
            curr = new_curr
            nearest_vertex = adj_mat.iloc[curr,1:].idxmin()#gives the minimum values index ie column
            nearest_vertex = int(nearest_vertex)
            if possible.iloc[nearest_vertex,0]==True: 
                dist = adj_mat.iloc[curr,nearest_vertex]
                distances.iloc[nearest_vertex,0] = dist
                possible.iloc[nearest_vertex,0]=False
                terminal_set.iloc[count,0] = nearest_vertex
                adj_mat.iloc[:,nearest_vertex] = inf
                new_curr = nearest_vertex
                count = count+1
        elif count == num_cities-1:
            curr = new_curr
            nearest_vertex = 0
            dist = adj_mat.iloc[curr,0]
            distances.iloc[0,0] = dist
            possible.iloc[0,0] = False
            terminal_set.iloc[count,0] = 0
    return terminal_set, distances
    

In [7]:
terminal_set, distances = nearest_neighbour(df_cityDist, num_cities, distances, possible, terminal_set)
display(terminal_set)
display(distances)

,terminal_set
0,3.0
1,4.0
2,1.0
3,2.0
4,0.0


,distances
0,100.0
1,4.0
2,18.0
3,14.0
4,7.0


In [8]:
adj_mat = df_cityDist.copy()
count = 0
new_curr = 0 #depot is the first vertex
#     while possible.iloc[0,0]== True: 
# if count < num_cities-1:
curr = new_curr
nearest_vertex = adj_mat.iloc[curr,1:].idxmin()#gives the minimum values index ie column
#     if possible.iloc[nearest_vertex,0]==True: 
#         dist = adj_mat.iloc[curr,nearest_vertex]
#         distances.iloc[nearest_vertex,0] = dist
#         possible.iloc[nearest_vertex,0]=False
#         terminal_set.loc[count,0] = nearest_vertex
#         adj_mat.iloc[:,nearest_vertex] = np.nan
#         new_curr = nearest_vertex
#         count = count+1
# elif count == num_cities-1:
# curr = new_curr
# nearest_vertex = 0
# dist = adj_mat.loc[curr,0]
# distaces.iloc[0,0] = dist
# possible.iloc[0,0] = False
# terminal_set.loc[count,0] = 0

In [9]:
adj_mat

,0,1,2,3,4
0,inf,inf,inf,inf,inf
1,16.000000,inf,inf,inf,inf
2,100.000000,inf,inf,inf,inf
3,14.000000,inf,inf,inf,inf
4,22.000000,inf,inf,inf,inf


In [10]:
curr

0

In [11]:
nearest_vertex =  int(nearest_vertex)

ValueError: cannot convert float NaN to integer

In [ ]:
adj_mat.iloc[nearest_vertex,0]